In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

def plot_mapa_calor_interativo(
    parquet_file,
    outlier_threshold_factor=3,
    data_inicio=None,
    data_fim=None,
    output_html="heatmap_plotly_correct.html"
):
    """
    Gera um mapa de calor interativo de notificações por setor censitário, filtrando por intervalo de datas,
    e tratando outliers.
    
    Args:
        parquet_file (str): Caminho para o arquivo Parquet.
        outlier_threshold_factor (float): Fator de multiplicação para determinar outliers.
        data_inicio (str): Data inicial no formato 'YYYYMM' (default: None, sem filtro inferior).
        data_fim (str): Data final no formato 'YYYYMM' (default: None, sem filtro superior).
        output_html (str): Caminho para salvar o arquivo HTML do mapa (default: "heatmap.html").
    """
    # Leitura do arquivo Parquet
    df = pd.read_parquet(parquet_file)

    # Garantir que a coluna 'mes_ano' é string para comparação
    df['mes_ano'] = df['mes_ano'].astype(str)

    # Filtro por intervalo de datas
    if data_inicio:
        df = df[df['mes_ano'] >= data_inicio]
    if data_fim:
        df = df[df['mes_ano'] <= data_fim]

    # Verifique se a coluna 'geometry' está corretamente interpretada
    df['geometry'] = gpd.GeoSeries.from_wkb(df['geometry'])

    # Converter para GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Agrupar os dados por setor censitário, somando as notificações
    gdf_agrupado = gdf.groupby('censitario').agg({
        'notificacoes': 'sum',
        'geometry': 'first'  # Preservar a geometria do setor censitário
    }).reset_index()

    # Criar um GeoDataFrame para visualização
    gdf_agrupado = gpd.GeoDataFrame(gdf_agrupado, geometry='geometry')

    # Calcular limites para identificação de outliers
    q1 = gdf_agrupado['notificacoes'].quantile(0.25)
    q3 = gdf_agrupado['notificacoes'].quantile(0.75)
    iqr = q3 - q1
    outlier_threshold = q3 + outlier_threshold_factor * iqr

    # Identificar outliers e criar uma coluna de flag
    gdf_agrupado['is_outlier'] = gdf_agrupado['notificacoes'] > outlier_threshold

    # Extrair coordenadas para o Plotly
    gdf_agrupado['lon'] = gdf_agrupado.geometry.centroid.x
    gdf_agrupado['lat'] = gdf_agrupado.geometry.centroid.y

    # Configurar o centro e o zoom do mapa com base nos setores censitários
    center_lat = gdf_agrupado['lat'].mean()
    center_lon = gdf_agrupado['lon'].mean()

    # Separar outliers e não outliers
    gdf_outliers = gdf_agrupado[gdf_agrupado['is_outlier']]
    gdf_normal = gdf_agrupado[~gdf_agrupado['is_outlier']]

    # Criar mapa interativo com Plotly para dados normais
    fig = px.scatter_mapbox(
        gdf_normal,
        lat='lat',
        lon='lon',
        color='notificacoes',
        size='notificacoes',
        size_max=15,
        mapbox_style="carto-positron",
        color_continuous_scale="Reds",
        hover_data={'notificacoes': True, 'censitario': True},
        title="Mapa de Calor - Notificações de Dengue em São José do Rio Preto",
        center=dict(lat=center_lat, lon=center_lon),
        zoom=12
    )
    
    # Definir o nome da primeira trace
    if len(fig.data) > 0:
        fig.data[0].name = 'Notificações'

    # Adicionar outliers com cor preta
    if not gdf_outliers.empty:
        outlier_trace = px.scatter_mapbox(
            gdf_outliers,
            lat='lat',
            lon='lon',
            size='notificacoes',
            size_max=15,
            mapbox_style="carto-positron",
            hover_data={'notificacoes': True, 'censitario': True}
        ).data[0]
        # Atualizar a cor dos outliers para preto
        outlier_trace.marker.color = 'black'
        outlier_trace.name = 'Outliers'
        # Ajustar propriedades adicionais se necessário
        outlier_trace.marker.sizemode = 'area'
        outlier_trace.marker.opacity = 0.7
        fig.add_trace(outlier_trace)

    # Atualizar layout para lidar com legendas
    fig.update_layout(
        legend=dict(
            title="Legenda",
            itemsizing='constant'
        )
    )

    # Salvar como HTML
    fig.write_html(output_html)
    print(f"Mapa salvo como {output_html}")

# Exemplo de uso:
parquet_file = "C:/Users/celso/Desktop/WIKI_DENGUE/lab-soft-wiki/encyclopedia/data_analytics/Sisaweb/notificacoes_count.parquet"

# Intervalo de datas para o filtro
data_inicio = "202201"  # Data inicial no formato YYYYMM
data_fim = "202212"     # Data final no formato YYYYMM

# Gerar o mapa
plot_mapa_calor_interativo(parquet_file, data_inicio=data_inicio, data_fim=data_fim)


In [5]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

def plot_mapa_calor_interativo(
    parquet_file,
    outlier_threshold_factor=3,
    data_inicio=None,
    data_fim=None,
    output_html="heatmap_plotly_optimized.html"
):
    """
    Gera um mapa de calor interativo de notificações por setor censitário, filtrando por intervalo de datas,
    e tratando outliers. HTML gerado é otimizado para ser menor.
    
    Args:
        parquet_file (str): Caminho para o arquivo Parquet.
        outlier_threshold_factor (float): Fator de multiplicação para determinar outliers.
        data_inicio (str): Data inicial no formato 'YYYYMM' (default: None, sem filtro inferior).
        data_fim (str): Data final no formato 'YYYYMM' (default: None, sem filtro superior).
        output_html (str): Caminho para salvar o arquivo HTML do mapa (default: "heatmap.html").
    """
    # Leitura do arquivo Parquet
    df = pd.read_parquet(parquet_file)

    # Garantir que a coluna 'mes_ano' é string para comparação
    df['mes_ano'] = df['mes_ano'].astype(str)

    # Filtro por intervalo de datas
    if data_inicio:
        df = df[df['mes_ano'] >= data_inicio]
    if data_fim:
        df = df[df['mes_ano'] <= data_fim]

    # Verificar e converter 'geometry' para GeoSeries
    df['geometry'] = gpd.GeoSeries.from_wkb(df['geometry'])

    # Converter para GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Agrupar os dados por setor censitário
    gdf_agrupado = gdf.groupby('censitario').agg({
        'notificacoes': 'sum',
        'geometry': 'first'
    }).reset_index()

    # Garantir que 'geometry' seja GeoSeries no GeoDataFrame resultante
    gdf_agrupado = gpd.GeoDataFrame(gdf_agrupado, geometry='geometry')

    # Calcular limites para outliers
    q1 = gdf_agrupado['notificacoes'].quantile(0.25)
    q3 = gdf_agrupado['notificacoes'].quantile(0.75)
    iqr = q3 - q1
    outlier_threshold = q3 + outlier_threshold_factor * iqr

    # Identificar outliers
    gdf_agrupado['is_outlier'] = gdf_agrupado['notificacoes'] > outlier_threshold

    # Extrair coordenadas e reduzir precisão
    gdf_agrupado['lon'] = gdf_agrupado.geometry.centroid.x.round(6)
    gdf_agrupado['lat'] = gdf_agrupado.geometry.centroid.y.round(6)

    # Configurar o centro do mapa
    center_lat = gdf_agrupado['lat'].mean()
    center_lon = gdf_agrupado['lon'].mean()

    # Separar outliers e dados normais
    gdf_outliers = gdf_agrupado[gdf_agrupado['is_outlier']]
    gdf_normal = gdf_agrupado[~gdf_agrupado['is_outlier']]

    # Criar mapa interativo para dados normais
    fig = px.scatter_mapbox(
        gdf_normal,
        lat='lat',
        lon='lon',
        color='notificacoes',
        size='notificacoes',
        size_max=15,
        mapbox_style="carto-positron",
        color_continuous_scale="Reds",
        hover_data={'notificacoes': True, 'censitario': True},
        title="Mapa de Calor - Notificações de Dengue",
        center=dict(lat=center_lat, lon=center_lon),
        zoom=12
    )
    
    # Definir o nome da primeira trace
    if len(fig.data) > 0:
        fig.data[0].name = 'Notificações'

    # Adicionar outliers com cor preta
    if not gdf_outliers.empty:
        fig.add_scattermapbox(
            lat=gdf_outliers['lat'],
            lon=gdf_outliers['lon'],
            mode='markers',
            marker=dict(size=10, color='black', opacity=0.7),
            name='Outliers',
            hoverinfo='skip'  # Remover hover para outliers
        )

    # Atualizar layout para legendas e margens menores
    fig.update_layout(
        legend_title="Legenda",
        margin=dict(l=10, r=10, t=30, b=10),
    )

    # Salvar como HTML otimizado
    fig.write_html(output_html, include_plotlyjs='cdn')  # Usar CDN para reduzir tamanho
    print(f"Mapa salvo como {output_html}")

# Exemplo de uso
parquet_file = "C:/Users/celso/Desktop/WIKI_DENGUE/lab-soft-wiki/encyclopedia/data_analytics/Sisaweb/notificacoes_count.parquet"
data_inicio = "202201"  # Data inicial no formato YYYYMM
data_fim = "202212"     # Data final no formato YYYYMM

# Gerar o mapa
plot_mapa_calor_interativo(parquet_file, data_inicio=data_inicio, data_fim=data_fim)


Mapa salvo como heatmap_plotly_optimized.html
